In [51]:
import pandas as pd
import numpy as np

In [125]:
newL = pd.read_csv('../data/resample_5000.csv')

In [126]:
db = pd.read_csv('../data/two_months.csv')
db = db[['userid','post_time','text']]

In [127]:
#merge with time variable 
time = pd.merge(newL, db, on='text')

In [128]:
print(time.shape)
time[1:5]


(3612, 18)


,userid_x,post_count,text,negative_emo,distortion,category,repost,negative_thinking,negative_event,emotion,Unnamed: 10,post_time_x,date_added_CESD,time_diff,Unnamed: 14,Unnamed: 15,userid_y,post_time_y
1,f8708568eff1535c85247b28dc126f9b,162.0,Fucking dance lift!,1,2,NaN,2,R,NaN,NaN,NaN,04/04/2011 16:47,10/05/2011 18:39,36,NaN,NaN,f8708568eff1535c85247b28dc126f9b,2011-04-04 16:47:33
2,3ac58ad39d81acfe169aadc4815effe6,62.0,It's frustrating when one little thing a perso...,1,1,negative filter,2,R,NaN,NaN,NaN,01/02/2011 19:37,11/02/2011 15:08,10,NaN,NaN,3ac58ad39d81acfe169aadc4815effe6,2011-02-01 19:37:30
3,5cc136477bfe3fcfe809dc451e4243ef,69.0,Is at one of those points where you just feel ...,1,1,overgeneralizing: entire life,2,R,NaN,NaN,NaN,03/03/2011 22:36,29/03/2011 10:20,26,NaN,NaN,5cc136477bfe3fcfe809dc451e4243ef,2011-03-03 22:36:08
4,0c5d06511ef79fba34da417e8c751fff,33.0,"I have Friday stuck in my head, and it WILL NO...",1,2,NaN,2,R,NaN,NaN,NaN,02/04/2011 0:51,15/04/2011 04:51,13,NaN,NaN,0c5d06511ef79fba34da417e8c751fff,2011-04-02 00:51:23


1 negative
2 positive
3 mixed
4 neutral
5 foreign language

In [6]:
time['negative_ny'].value_counts()


2    1643
1    1279
4     708
3     212
5      39
Name: negative_ny, dtype: int64

percentage of negative post in each user

In [33]:
#select negative post then groupby
neg_g = time.loc[time['negative_ny'] == 1].groupby(['userid'])
#negative post count
neg_c = neg_g['negative_ny'].count().reset_index()
neg_c.columns = ['userid', 'neg_count']

#all post count
post_c = time.groupby(['userid']).count().negative_ny.reset_index()
post_c.columns = ['userid', 'all_count']

per_df = pd.merge(neg_c, post_c, how = 'right', on = 'userid')
per_df['neg_count'].fillna(0, inplace=True)
#negative post percentage
per_df['neg_per'] = per_df['neg_count']/per_df['all_count']
per_df = per_df.drop('neg_count', axis=1)
per_df[1:10]

,userid,all_count,neg_per
1,00c69df4ec41f61da30ed2874b9fbfaf,1,1.000000
2,011679a3ae49998801ada93d822b7302,11,0.454545
3,01530624766b3be63c7709c6581c335d,12,0.583333
4,04a3601e8b319f740167723a58c488e7,14,0.285714
5,04be1d6fa64adb60a3d2847f4f905ec1,15,0.533333
6,054e5fd22c26a92926a61c261c99054c,9,0.222222
7,055638f0a33b2df4031fadb86076861f,21,0.095238
8,059c18ea701ce88479c9e10a25cb8284,7,0.285714
9,05a1a1b67e3d7a923f9d0ee5502757c0,6,0.666667


In [24]:
def count_percentage(category, OutputCol, OutputDf):
    cat_g = category.groupby(['userid'])
    #negative post count
    cat_c = cat_g['negative_ny'].count().reset_index()
    cat_c.columns = ['userid', 'cat_count']
    per_df = pd.merge(cat_c,OutputDf, how = 'right', on = 'userid')
    per_df['cat_count'].fillna(0, inplace=True)
    #negative post percentage
    per_df[OutputCol] = per_df['cat_count']/per_df['all_count']
    per_df = per_df.drop('cat_count', axis=1)
    return per_df

percentage of positive post in each user

In [34]:
cat_pos = time.loc[time['negative_ny'] == 2]
per_df = count_percentage(cat_pos,'pos_per', per_df)


percentage of mixed post in each user

In [35]:
cat_mix = time.loc[time['negative_ny'] == 3]
per_df = count_percentage(cat_mix,'mix_per', per_df)


percentage of neutral post in each user

In [67]:
#select mixed post then groupby
cat_neu = time.loc[time['negative_ny'] == 4]
per_df = count_percentage(cat_neu,'neu_per', per_df)
print(per_df.shape)
per_df[1:10]

(717, 6)


,userid,all_count,neg_per,pos_per,mix_per,neu_per
1,011679a3ae49998801ada93d822b7302,11,0.454545,0.454545,0.000000,0.090909
2,01530624766b3be63c7709c6581c335d,12,0.583333,0.333333,0.000000,0.083333
3,0467ce1e63e2a48da92e1ff53f052829,2,0.000000,0.500000,0.000000,0.500000
4,04a3601e8b319f740167723a58c488e7,14,0.285714,0.500000,0.142857,0.071429
5,04be1d6fa64adb60a3d2847f4f905ec1,15,0.533333,0.400000,0.000000,0.066667
6,055638f0a33b2df4031fadb86076861f,21,0.095238,0.714286,0.095238,0.095238
7,0765c56b2da0e50568488dbbbe93751b,8,0.625000,0.125000,0.000000,0.250000
8,077243dfd4750094bde7eb01e5cfcb42,1,0.000000,0.000000,0.000000,1.000000
9,07da88e13509940f3116716552d6a32c,8,0.125000,0.500000,0.000000,0.375000


In [ ]:
#match ces-d,big5 and swl

In [37]:
scale = pd.read_csv('../data/users_to_be_analysed.csv')
scale.columns

In [76]:
post_c = time.groupby(['userid']).count().negative_ny.reset_index()
post_c.columns = ['userid', 'all_count']
post_c.shape
post_c2 = post_c.drop_duplicates(subset='userid', keep='first', inplace=False)
post_c2.shape
post_c2.to_csv('text.csv')

In [129]:
two = pd.read_csv('../data/resample_5000.csv')
two = two.drop_duplicates(subset='userid', keep='first', inplace=False)
two.shape

(78, 16)

In [42]:
scale = scale[['userid','swl','CESD_sum','ope','con','ext','agr','neu']]
scale[1:5]

,userid,swl,CESD_sum,ope,con,ext,agr,neu
1,0724fe854bd455061ba84efecdeff469,6.4,6,4.8,3.55,3.00,2.95,3.15
2,088bf0ed841bcc366475b3975eb1b9d2,2.4,48,4.5,2.10,1.90,2.60,4.65
3,0b12516100c16f779b152858619786ff,5.6,30,4.0,3.80,2.55,2.75,4.20
4,0e3fdb89c7706b13891170ab2a9017c7,6.0,4,4.8,3.15,2.85,4.50,1.85


In [56]:
all_df = pd.merge(per_df, scale, how = 'right', on = 'userid')
all_df = all_df[np.isfinite(all_df['all_count'])]
print(all_df.shape)
all_df[1:5]

(85, 13)


,userid,all_count,neg_per,pos_per,mix_per,neu_per,swl,CESD_sum,ope,con,ext,agr,neu
1,088bf0ed841bcc366475b3975eb1b9d2,18.0,0.333333,0.333333,0.055556,0.277778,2.4,48,4.50,2.10,1.90,2.60,4.65
2,0e3fdb89c7706b13891170ab2a9017c7,16.0,0.187500,0.687500,0.000000,0.125000,6.0,4,4.80,3.15,2.85,4.50,1.85
3,193165601dfad67e79ef52b2caf9fd9e,4.0,0.000000,0.750000,0.000000,0.250000,5.8,10,4.00,3.50,4.25,4.05,2.90
4,1c1a61d11a10f9fa967f250095792db3,10.0,0.100000,0.700000,0.000000,0.200000,4.8,41,4.55,2.40,3.35,2.65,2.70


In [58]:
all_df.describe()

,all_count,neg_per,pos_per,mix_per,neu_per,swl,CESD_sum,ope,con,ext,agr,neu
count,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000
mean,9.764706,0.303694,0.523150,0.029007,0.137362,4.049412,24.329412,4.171059,3.190353,3.120353,3.517647,2.998588
std,9.176376,0.233077,0.262147,0.065748,0.202033,1.492172,12.116879,0.485141,0.729067,0.890160,0.666354,0.873294
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.400000,2.000000,2.950000,1.500000,1.300000,1.600000,1.050000
25%,4.000000,0.142857,0.384615,0.000000,0.000000,2.800000,15.000000,3.800000,2.750000,2.450000,3.200000,2.400000
50%,7.000000,0.297297,0.500000,0.000000,0.052632,4.000000,25.000000,4.170000,3.200000,3.000000,3.550000,3.000000
75%,11.000000,0.428571,0.700000,0.000000,0.230769,5.400000,34.000000,4.600000,3.750000,3.850000,3.900000,3.550000
max,50.000000,1.000000,1.000000,0.400000,1.000000,6.800000,48.000000,4.950000,4.600000,4.800000,4.850000,4.650000


In [66]:
all_df2 = all_df.drop_duplicates(subset='userid', keep='first', inplace=False)
all_df2.shape

(85, 13)